# How to load data to BigQuery

Here we demonstrate a few different ways to load data to BigQuery from an R notebook.
* [bq](https://cloud.google.com/bigquery/docs/bq-command-line-tool) command line tool
* [bigrquery](https://cloud.google.com/blog/products/gcp/google-cloud-platform-for-data-scientists-using-r-with-google-bigquery-part-2-storing-and-retrieving-data-frames)

<div class="alert alert-block alert-info">
<b>Tip:</b> See also the companion Terra Support article <a href='https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-Advanced-GCP-features-in-Terra'>Accessing Advanced GCP features in Terra</a>.
</div>

## Setup

Edit the global variables in your clone of this notebook to refer to a native Google Cloud Platform project to which you have WRITE acces.
* **The destination BigQuery dataset should already exist. Your pet account must have WRITE access to it.**       
[**Click for step-by-step instructions to create a BQ dataset**](https://support.terra.bio/hc/en-us/articles/360051229072#h_01EPCCS08S69VE4VMT0F0NNDWR)     


* Make sure to change to your own project and dataset names. The remaining cells can be run as-is.

In [1]:
library(jsonlite)
library(bigrquery)
library(lubridate)
library(tidyverse)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ purrr::flatten()         masks jsonlite::flatten()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()



In [2]:
# This file loads fine via autodetect.
CSV_PATH <- 'gs://genomics-public-data/platinum-genomes/other/platinum_genomes_sample_info.csv'

# Also try this CSV which will yield some autodetect errors.
CSV_PATH_AUTODETECT_FAILS <- 'gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv'

BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

**Note that you will need to change the variables below to your own values** (expand the tips if you need help finding the variables)

In [3]:
# CHANGE THESE VARIABLES
DESTINATION_PROJECT_ID <- 'your_GCP-native_project_ID'
DESTINATION_DATASET <- 'your_BQ_dataset'

In [4]:
DESTINATION_PROJECT_ID <- 'ah-native-gcp-project-74939'
DESTINATION_DATASET <- 'BQ_dataset_autodelete_after_one_day'

### <font color="#FF6600">(expand for tip) </font> <font color="#445555">How to find your cloud-native project-ID</font><a class="tocSkip">

When logged in with your Terra user-ID, go to billing in the GCP console at [https://console.cloud.google.com/billing](https://console.cloud.google.com/billing)     
![finding project ID screen shot](https://storage.googleapis.com/terra-featured-workspaces/QuickStart/Advanced-GCP-features_Find-Project-ID_Step1_Screen%20shot.png)

1. Select the Organization you used when creating your cloud-native project    
2. Find the Project ID at right  

### <font color="#FF6600">(expand for tip) </font> <font color="#445555">How to find your cloud-native BigQuery dataset</font><a class="tocSkip">

Go to [https://console.cloud.google.com/bigquery](https://console.cloud.google.com/bigquery)   

On the left column, select your cloud-native Project from the drop-down. You should see your BQ dataset listed:   

![Find BQ dataset Screen shiot](https://storage.googleapis.com/terra-featured-workspaces/QuickStart/Advanced-GCP-features_Find-BQ-dataset-name_Screen%20shot.png)

# Load data to BigQuery from a CSV

We'll do this using the `bq` command line tool and the `--autodetect` flag.

In [5]:
DESTINATION_TABLE <- paste0('r_bq_autodetect_', strftime(now(), '%Y%m%d_%H%M%S'))

In [6]:
system(str_glue(str_c('bq --project_id {BILLING_PROJECT_ID} load ',
                      '--autodetect ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE} ',
                      '{CSV_PATH}  2>&1')),
      intern = TRUE)

[1] "\rWaiting on bqjob_r6a7a7e8c974b205b_0000017986fc45b2_1 ... (0s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r6a7a7e8c974b205b_0000017986fc45b2_1 ... (1s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r6a7a7e8c974b205b_0000017986fc45b2_1 ... (1s) Current status: DONE   "

Show the table schema.

In [7]:
system(str_glue(str_c('bq --project_id {BILLING_PROJECT_ID} show ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE}')),
      intern = TRUE)

[1] "Table ah-native-gcp-project-74939:BQ_dataset_autodelete_after_one_day.r_bq_autodetect_20210519_233548"                                                
 [2] ""                                                                                                                                                     
 [3] "   Last modified                  Schema                 Total Rows   Total Bytes     Expiration      Time Partitioning   Clustered Fields   Labels  "
 [4] " ----------------- ------------------------------------ ------------ ------------- ----------------- ------------------- ------------------ -------- "
 [5] "  19 May 23:35:52   |- Catalog_ID: string                17           1441          20 May 23:35:52                                                  "
 [6] "                    |- Description: string                                                                                                           "
 [7] "                    |- Gender: string                                                                                                                "
 [8] "                    |- Race: string                                                                                                                  "
 [9] "                    |- Family_Member: integer                                                                                                        "
[10] "                    |- Relationship_to_Proband: string                                                                                               "
[11] "                    |- CT_Desc: string                                                                                                               "
[12] "                    |- Cum_Pdl: string                                                                                                               "
[13] ""

# Load data to BigQuery from a dataframe

We'll do this using R package `bigrquery`.

In [8]:
DESTINATION_TABLE <- paste0('r_bigrquery_', strftime(now(), '%Y%m%d_%H%M%S'))

In [9]:
insert_upload_job(project = DESTINATION_PROJECT_ID,
                  dataset = DESTINATION_DATASET,
                  table = DESTINATION_TABLE,
                  billing = BILLING_PROJECT_ID,
                  write_disposition = 'WRITE_EMPTY',
                  mtcars)

Warning message:
“'insert_upload_job' is deprecated.
Use 'bq_perform_upload' instead.
See help("Deprecated") and help("bigrquery-deprecated").”


$kind
[1] "bigquery#job"

$etag
[1] "maQ+6+B5YU0LnJBoo6gFew=="

$id
[1] "fc-product-demo:US.job_zSfBB2sbfA3AyhteotIa87Cz5ik7"

$selfLink
[1] "https://www.googleapis.com/bigquery/v2/projects/fc-product-demo/jobs/job_zSfBB2sbfA3AyhteotIa87Cz5ik7?location=US"

$user_email
[1] "pet-110502112294304402890@fc-product-demo.iam.gserviceaccount.com"

$configuration
$configuration$load
$configuration$load$schema
$configuration$load$schema$fields
$configuration$load$schema$fields[[1]]
$configuration$load$schema$fields[[1]]$name
[1] "mpg"

$configuration$load$schema$fields[[1]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[1]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[1]]$description
[1] ""


$configuration$load$schema$fields[[2]]
$configuration$load$schema$fields[[2]]$name
[1] "cyl"

$configuration$load$schema$fields[[2]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[2]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[2]]$description
[1] ""


$configuration$load$schema$fields[[3]]
$configuration$load$schema$fields[[3]]$name
[1] "disp"

$configuration$load$schema$fields[[3]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[3]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[3]]$description
[1] ""


$configuration$load$schema$fields[[4]]
$configuration$load$schema$fields[[4]]$name
[1] "hp"

$configuration$load$schema$fields[[4]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[4]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[4]]$description
[1] ""


$configuration$load$schema$fields[[5]]
$configuration$load$schema$fields[[5]]$name
[1] "drat"

$configuration$load$schema$fields[[5]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[5]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[5]]$description
[1] ""


$configuration$load$schema$fields[[6]]
$configuration$load$schema$fields[[6]]$name
[1] "wt"

$configuration$load$schema$fields[[6]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[6]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[6]]$description
[1] ""


$configuration$load$schema$fields[[7]]
$configuration$load$schema$fields[[7]]$name
[1] "qsec"

$configuration$load$schema$fields[[7]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[7]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[7]]$description
[1] ""


$configuration$load$schema$fields[[8]]
$configuration$load$schema$fields[[8]]$name
[1] "vs"

$configuration$load$schema$fields[[8]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[8]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[8]]$description
[1] ""


$configuration$load$schema$fields[[9]]
$configuration$load$schema$fields[[9]]$name
[1] "am"

$configuration$load$schema$fields[[9]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[9]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[9]]$description
[1] ""


$configuration$load$schema$fields[[10]]
$configuration$load$schema$fields[[10]]$name
[1] "gear"

$configuration$load$schema$fields[[10]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[10]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[10]]$description
[1] ""


$configuration$load$schema$fields[[11]]
$configuration$load$schema$fields[[11]]$name
[1] "carb"

$configuration$load$schema$fields[[11]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[11]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[11]]$description
[1] ""




$configuration$load$destinationTable
$configuration$load$destinationTable$projectId
[1] "ah-native-gcp-project-74939"

$configuration$load$destinationTable$datasetId
[1] "BQ_dataset_autodelete_after_one_day"

$configuration$load$destinationTable$tableId
[1] "r_bigrquery_20210519_233555"


$configuration$load$createDisposition
[1] "CREATE_IF_NEEDED"

$configuration$load$writeDisposition
[1] "WRITE_EMPTY"

$configuration$load$sourceFormat
[1] "NEWLINE_DELIMITED_JSON"


$configuration$jobType
[1] "LOAD"


$jobReference
$jobReference$projectId
[1] "fc-product-demo"

In [10]:
# Create a "connection" to a public BigQuery dataset.
dbcon <- bigrquery::src_bigquery(project = DESTINATION_PROJECT_ID,
                                 dataset = DESTINATION_DATASET,
                                 billing = BILLING_PROJECT_ID)

# Create a 'virtual dataframe' backed by a BigQuery table.
tbl <- dplyr::tbl(dbcon, DESTINATION_TABLE)
colnames(tbl)

[1] "mpg"  "cyl"  "disp" "hp"   "drat" "wt"   "qsec" "vs"   "am"   "gear"
[11] "carb"

# Provenance

In [11]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.5 (2021-03-31)
 os       Ubuntu 18.04.5 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2021-05-19                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version date       lib source        
 assertthat    0.2.1   2019-03-21 [2] CRAN (R 4.0.5)
 backports     1.2.1   2020-12-09 [2] CRAN (R 4.0.5)
 base64enc     0.1-3   2015-07-28 [2] CRAN (R 4.0.5)
 bigrquery   * 1.3.2   2020-10-05 [2] CRAN (R 4.0.5)
 bit           4.0.4   2020-08-04 [2] CRAN (R 4.0.5)
 bit64         4.0.5   2020-08-30 [2] CRAN (R 4.0.5)
 broom         0.7.6   2021-04-05 [2] CRAN (R 4.0.5)
 cachem        1.0.4   2021

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.